In [1]:
from ampel.contrib.hu.examples.t2.T2ExamplePolyFit import T2ExamplePolyFit
import pickle

Pickle load a lightcurve object (a lightcurve instance be provided as input of any T2Runnable unit)

In [2]:
lc = pickle.load(open("../src/ampel/contrib/hu/examples/t2/lightcurve.pickle", "rb"))

In [3]:
lc.get_values("mag")

[23.856793629352122,
 23.00805766234931,
 21.595161397095065,
 23.447385814760864,
 23.064335890370327,
 21.117456462709832,
 21.69892544131502,
 20.54556957554393,
 20.881180998046148,
 20.340208010886663,
 20.187509359597723,
 20.40742267815947,
 20.588474220085388,
 20.556809476480602]

Instanciate example t2 class

In [4]:
t2_runnable = T2ExamplePolyFit()

A T2 class *must* inherit ```ampel.abstract.AbstractT2Runnable``` and implement the two functions:
- ```run(self, light_curve, run_parameters)```
- ```get_version(self)```

(otherwise exception will be throwed).

The function ```set_base_parameters(self, dict_instance)``` - if implemented - will be called automatically by Ampel after class instanciation.

The ampel config entry for ```T2ExamplePolyFit``` could look like this:

```json
{
  'T2': {
       'runnables' : {
           'POLYFIT' : {
              'classFullPath': 'ampel.contrib.hu.examples.t2.T2ExamplePolyFit',
               'base_parameters': {
                  'fit_function' : 'polyfit',
               },
               'run_parameters' : {
                   'default' : {
                      'degree' : 3
                   },
                   'advanced' : {
                      'degree' : 5
                   }
               }
           }
       }
   }
}
```

The following method is called automatically by the Ampel T2 controller after the t2runnable class instanciation (based on the ampel config)

In [5]:
t2_runnable.set_base_parameters(
    {"fit_function": "polyfit"}
)

The Ampel T2 controller will call the method ```run``` and provide an ampel.base.LightCurve instance and the dict instance  ```run_parameters```

In [6]:
result = t2_runnable.run(
    light_curve=lc, 
    run_parameters={'degree': 3}
)
result

2018-02-23 12:06:24 T2ExamplePolyFit.py:74 run() INFO Please use 'self.logger' for logging
2018-02-23 12:06:24 T2ExamplePolyFit.py:75 run() DEBUG By doing that, log entries will be automatically recorded into the database!


{'chi2': 4.276766382894305,
 'polyfit': [0.00010168208142339463,
  -17.710519722219509,
  1028245.6388793393,
  -19899452971.043541]}

T2Controller will automatically save the dict returned by ```run``` into the Ampel database<br>
Please note that T2Controller will also complete the dictionary with the following entries:
- ```version```: the version number returned by get_version() of the T2ExamplePolyFit instance
- ```run_parameters```: the run parameters provided to the method ```run``` (originating from the value of ```paramId``` in the corresponding T2 database document)


----------
Note: the method ```run``` must return a BSON encodable dictionnary.<br>
If you look at the source code of ```run```, you'll see that the  numpy array outputed by polyfit is casted into a python list. This is because a numpy array is not BSON encodable:<br><font color="red"> (following error and traceback are created on purpose)</font>

In [7]:
import bson, numpy
bson.BSON.encode({'a': numpy.array([1,2,3])})

InvalidDocument: Cannot encode object: array([1, 2, 3])

whereas a python list is:

In [8]:
bson.BSON.encode({'a': [1,2,3]})

b'"\x00\x00\x00\x04a\x00\x1a\x00\x00\x00\x100\x00\x01\x00\x00\x00\x101\x00\x02\x00\x00\x00\x102\x00\x03\x00\x00\x00\x00\x00'